# Pre processing

In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession.builder.appName('Spaceship').getOrCreate()

In [17]:
train_data = spark.read.csv("train.csv", inferSchema=True, header=True)
validation_data = spark.read.csv("test.csv", inferSchema=True, header=True)

train_data.show()

+-----------+----------+---------+-----+-------------+----+-----+-----------+---------+------------+------+------+------------------+-----------+
|PassengerId|HomePlanet|CryoSleep|Cabin|  Destination| Age|  VIP|RoomService|FoodCourt|ShoppingMall|   Spa|VRDeck|              Name|Transported|
+-----------+----------+---------+-----+-------------+----+-----+-----------+---------+------------+------+------+------------------+-----------+
|    0001_01|    Europa|    false|B/0/P|  TRAPPIST-1e|39.0|false|        0.0|      0.0|         0.0|   0.0|   0.0|   Maham Ofracculy|      false|
|    0002_01|     Earth|    false|F/0/S|  TRAPPIST-1e|24.0|false|      109.0|      9.0|        25.0| 549.0|  44.0|      Juanna Vines|       true|
|    0003_01|    Europa|    false|A/0/S|  TRAPPIST-1e|58.0| true|       43.0|   3576.0|         0.0|6715.0|  49.0|     Altark Susent|      false|
|    0003_02|    Europa|    false|A/0/S|  TRAPPIST-1e|33.0|false|        0.0|   1283.0|       371.0|3329.0| 193.0|      Sola

In [20]:
fillna_values = {'CryoSleep': False, 
                  'VIP': False,
                  'Age': 0,
                  'VRDeck': 0,
                  'Cabin': '/0/',
                  'Destination': '',
                  'ShoppingMall': 0,
                  'Name': '',
                  'RoomService': 0,
                  'FoodCourt': 0,
                  'HomePlanet': '',
                  'Spa': 0}

validation_data = validation_data.fillna(value=fillna_values)
train_data = train_data.fillna(value=fillna_values)
train_data.show()

+-----------+----------+---------+-----+-------------+----+-----+-----------+---------+------------+------+------+------------------+-----------+
|PassengerId|HomePlanet|CryoSleep|Cabin|  Destination| Age|  VIP|RoomService|FoodCourt|ShoppingMall|   Spa|VRDeck|              Name|Transported|
+-----------+----------+---------+-----+-------------+----+-----+-----------+---------+------------+------+------+------------------+-----------+
|    0001_01|    Europa|    false|B/0/P|  TRAPPIST-1e|39.0|false|        0.0|      0.0|         0.0|   0.0|   0.0|   Maham Ofracculy|      false|
|    0002_01|     Earth|    false|F/0/S|  TRAPPIST-1e|24.0|false|      109.0|      9.0|        25.0| 549.0|  44.0|      Juanna Vines|       true|
|    0003_01|    Europa|    false|A/0/S|  TRAPPIST-1e|58.0| true|       43.0|   3576.0|         0.0|6715.0|  49.0|     Altark Susent|      false|
|    0003_02|    Europa|    false|A/0/S|  TRAPPIST-1e|33.0|false|        0.0|   1283.0|       371.0|3329.0| 193.0|      Sola

In [29]:
print(train_data.count())
print(validation_data.count())

8693
4277


### Create deck, num, side columns

In [30]:
from pyspark.sql.functions import split

split_col = split(train_data['Cabin'], '/')
train_data = train_data.withColumn('Deck', split_col.getItem(0)) \
       .withColumn('Num', split_col.getItem(1)) \
       .withColumn('Side', split_col.getItem(2))

split_col = split(validation_data['Cabin'], '/')
validation_data = validation_data.withColumn('Deck', split_col.getItem(0)) \
       .withColumn('Num', split_col.getItem(1)) \
       .withColumn('Side', split_col.getItem(2))

### Create passenger group column